In [251]:
import json
import pandas as pd
#!pip install openai -U
from openai import OpenAI
from enum import Enum
from typing import Optional, List
from pydantic import BaseModel
from pydantic import Field

In [252]:
with open('../key/key.json') as f:
    k = json.load(f)['key']

In [256]:
# create one time list of 10 human emotions
client = OpenAI(api_key=k)
class Chr_answer(BaseModel):
    Characteristics: List[str] = Field(None, description="Emotions as a list of strings")

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Be a helpful assistant."},
        {"role": "user", "content": "List 30 human emotions that describe diffnon redundant human feelings."}
    ],
    response_format=Chr_answer,
)

#you can also procees from parsed eg json.loads(completion_song.choices[0].message.parsed.json())
out = json.loads(completion.choices[0].message.content)
out = list(out.values())[0]
emotions = out
print(emotions)

['Joy', 'Sadness', 'Anger', 'Fear', 'Surprise', 'Disgust', 'Trust', 'Anticipation', 'Love', 'Hate', 'Shame', 'Pride', 'Jealousy', 'Contentment', 'Nostalgia', 'Gratitude', 'Loneliness', 'Embarrassment', 'Envy', 'Guilt', 'Curiosity', 'Awe', 'Boredom', 'Skepticism', 'Relief', 'Frustration', 'Despair', 'Peace', 'Amusement', 'Determination']


In [268]:
# class Char(BaseModel): 
#     is_char: bool
#     characteristic: Optional[Characteristic] ?
#fiels is only defined for the first custom class?

#List 10 best selling American clothing brands and, for each brand, list which one of the emotions in the given list are most closely associated with."
client = OpenAI(api_key=k)

# class Characteristic(Enum):
#     Happiness = 'Happiness'
#     Sadness = 'Sadness'
#     Anger = 'Anger'
#     Fear = 'Fear'
#     Surprise = 'Surprise'
#     Disgust = 'Disgust'
#     Trust = 'Trust'
#     Anticipation ='Anticipation'
#     Joy ='Joy'
#     Love = 'Love'

# class BrandAttributes(BaseModel):
#     brand_name: str = Field(None, description=" name as str")
#     characteristics: List[Characteristic] = Field(None, description=" list of characteristics as str")

Characteristic = Enum('Characteristic', dict([(i, i) for i in emotions])) #MyEnumType = Enum('MyEnumType', myEnumStrings)

class BrandAttributes(BaseModel):
    brand_name: str = Field(None, description=" name as str")
    characteristics: List[Characteristic] = Field(None, description=" list of characteristics as str")

class BrandChars(BaseModel):
    a: List[BrandAttributes] = Field(None, description="List of BrandAttributes")
    
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Be a helpful assistant."},
        {"role": "user", "content": "List 30 best selling American clothing brands and, for each brand, list which one of the emotions in the given list are most closely associated with."}
    ],
    response_format=BrandChars,
)

out = json.loads(completion.choices[0].message.content)
brand_emotions = out
out = list(out.values())[0]
df_brand = pd.DataFrame(data = [i.values() for i in out], columns = ['Name','characteristic'])
df_brand

,Name,characteristic
0,Nike,"[Determination, Trust]"
1,Levi's,"[Nostalgia, Contentment]"
2,Victoria's Secret,"[Love, Pride]"
3,Ralph Lauren,"[Pride, Trust]"
4,Tommy Hilfiger,"[Nostalgia, Pride]"
5,Under Armour,"[Determination, Trust]"
6,Calvin Klein,"[Love, Pride]"
7,Gap,"[Nostalgia, Trust]"
8,American Eagle Outfitters,"[Contentment, Trust]"
9,Abercrombie & Fitch,"[Pride, Nostalgia]"


In [269]:
#Take the example song, and list which one of the emotions in the given list are most closely associated with the song.")

client = OpenAI(api_key=k)

Characteristic = Enum('Characteristic', dict([(i, i) for i in emotions])) 

class SongAttributes(BaseModel):
    song_name: str
    song_lyrics_summary :str = Field(None, description="Summary as str; Don't state the exact lyrics.")
    characteristics: List[Characteristic]

class SongsChars(BaseModel):
    a: List[SongAttributes] = Field(None, description="List of song attributes.")

prompt = """
As prompt, take the song 'Black Balloon', list summary of its lyrics, and a list of the emotions from the given list that are most closely associated with the song.
"""

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Be a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    response_format=SongsChars,
)
out = json.loads(completion.choices[0].message.content)
out = list(out.values())[0]
df_song = pd.DataFrame(data = [i.values() for i in out], columns = ['Name','lyrics_summary', 'characteristic'])
df_song

,Name,lyrics_summary,characteristic
0,Black Balloon,The lyrics of 'Black Balloon' explore themes o...,"[Sadness, Love, Despair, Frustration, Nostalgia]"


In [272]:
#MultilabelBinarize brands and song
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(classes=emotions)

df_brand2 = pd.DataFrame(mlb.fit_transform(df_brand['characteristic']), columns = emotions)
df_brand_mb = pd.merge(df_brand2, df_brand[['Name']], how ='left', left_index=True, right_index=True)
df_brand_mb.head()


,Joy,Sadness,Anger,Fear,Surprise,Disgust,Trust,Anticipation,Love,Hate,...,Awe,Boredom,Skepticism,Relief,Frustration,Despair,Peace,Amusement,Determination,Name
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,Nike
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Levi's
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Victoria's Secret
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Ralph Lauren
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Tommy Hilfiger


In [273]:
mlb = MultiLabelBinarizer(classes=emotions)

df_song2 = pd.DataFrame(mlb.fit_transform(df_song['characteristic']), columns = emotions)
df_song_mb = pd.merge(df_song2, df_song[['Name']], how ='left', left_index=True, right_index=True)
df_song_mb.head()

#song_e = list(song_emotions[0].values())[2]

#df_song = pd.DataFrame(mlb.fit_transform([song_e]), columns = emotions, index=[list(song_emotions[0].keys())[0]])
#df_song

,Joy,Sadness,Anger,Fear,Surprise,Disgust,Trust,Anticipation,Love,Hate,...,Awe,Boredom,Skepticism,Relief,Frustration,Despair,Peace,Amusement,Determination,Name
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,1,0,0,0,Black Balloon


In [281]:
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split 
  
# Create feature and target arrays 
X = df_brand_mb.drop(['Name'], axis =1)
y = df_brand_mb['Name'] 
  
# Split into training and test set 
X_train, X_test, y_train, y_test = train_test_split( 
             X, y, test_size = 0.2, random_state=42) 
  
knn = KNeighborsClassifier(n_neighbors=3) 
  
knn.fit(X_train, y_train) 
  
# Predict on dataset which model has not seen before 
print(knn.predict(X_test.values))

['Nike' 'Dockers' 'Express' 'Hollister' 'Dockers' 'Guess by Marciano']


/Users/neda/opt/anaconda3/envs/ds/lib/python3.8/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [282]:
X_test

,Joy,Sadness,Anger,Fear,Surprise,Disgust,Trust,Anticipation,Love,Hate,...,Curiosity,Awe,Boredom,Skepticism,Relief,Frustration,Despair,Peace,Amusement,Determination
27,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
15,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [283]:
#take a look
df_brand_mb[df_brand_mb['Name'].isin(list(knn.predict(X_test.values)))]

/Users/neda/opt/anaconda3/envs/ds/lib/python3.8/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


,Joy,Sadness,Anger,Fear,Surprise,Disgust,Trust,Anticipation,Love,Hate,...,Awe,Boredom,Skepticism,Relief,Frustration,Despair,Peace,Amusement,Determination,Name
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,Nike
22,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Hollister
24,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Dockers
28,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Guess by Marciano
29,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Express


In [ ]:
#finding an optimal text embedding:
#First embed them since used by most methods
# Here, we do need to know the frequency for each of the emotions
# we have a small dataset

In [ ]:


embeddings = [nlp(sentence).vector for sentence in sentences]
distance = euclidean_distance(embeddings[0], embeddings[1])
print(distance)

# OUTPUT
1.8646982721454675

A = ['satisfied','happy']
B= ['Sadness', 'Love', 'Trust']
def jaccard_similarity(x,y):
  """ returns the jaccard similarity between two lists """
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)

print('Jaccard Similarity:', jaccard_similarity(A,B))

In [ ]:

#OpenAI example
#define 3 categories to be represented to the user as strings in json
#class Category(Enum):
#    violence = "violence"
#    sexual = "sexual"
#    self_harm = "self_harm"

#defining what my data type looks like so it properly imported and exported as json
#class ContentCompliance(BaseModel): 
#    is_violating: bool  #tell me if there is a violation and if so, classify which one of the category classes
#    category: Optional[Category] #the type of the category could be none or category. If non violating, no need to define category
#    explanation_if_violating: Optional[str]
